In [1]:
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 3 Aug 28 13:56 /dev/video1
crw-rw---- 1 root video 81, 0 Aug 28 13:56 /dev/video0


In [2]:
from jetcam.csi_camera import CSICamera

class MyCamera(CSICamera):
    def __init__(self, *args, **kwargs):
        super(MyCamera, self).__init__(*args, **kwargs)

    ## https://forums.developer.nvidia.com/t/example-gstreamer-pipeline-to-test-li-imx219-mipi-ff-nano-on-the-jetson-nano/72226
    # gst-launch-1.0 nvarguscamerasrc ! ‘video/x-raw(memory:NVMM),width=1024, height=768, framerate=120/1, format=NV12’ ! nvvidconv flip-method=0 ! nvegltransform ! nveglglessink -e
    def _gst_str(self):
        return 'nvarguscamerasrc sensor-id=%d ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv flip-method=0 !  nvjpegenc ! appsink' % (
                self.capture_device, self.capture_width, self.capture_height, self.capture_fps) # , self.width, self.height
    
    def release(self):
        self.cap.release()

In [3]:
# https://www.waveshare.com/wiki/IMX219-83_Stereo_Camera
# Resolution: 3280 × 2464 (per camera)
WIDTH = 3280//8
HEIGHT = 2646//8

try: cameras
except NameError:
    cameraLeft = MyCamera(capture_device=0, capture_width=WIDTH, capture_height=HEIGHT)
    cameraRight = MyCamera(capture_device=1, capture_width=WIDTH, capture_height=HEIGHT)

    cameras = [cameraLeft, cameraRight]
    
display(cameras)

[<__main__.MyCamera at 0x7f7f641ef0>, <__main__.MyCamera at 0x7fa4e376d8>]

In [4]:
def transform_image(image):
    return image.flatten().tobytes()

In [5]:
import ipywidgets
from IPython.display import display

try: widgets
except NameError:
    widgets = []
    
if not widgets:
    for camera in cameras:
        camera.running = False

        image = transform_image(camera.read())

        image_widget = ipywidgets.Image(format='jpeg')
        image_widget.value = image
        display(image_widget)

        widgets.append(image_widget)
        
display(widgets)

Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08…

Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08…

[Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07...', format='jpeg'),
 Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07...', format='jpeg')]

In [6]:
from functools import partial

def update_image(image_widget, change):
    image = transform_image(change['new'])
    image_widget.value = image

try: observers
except NameError:
    observers = []
    
if len(observers) < len(widgets):
    for (camera, widget) in zip(cameras, widgets):
        observer = partial(update_image, widget)
        camera.observe(observer, names='value')

        camera.running = True

        observers.append(observer)

display(observers)

[functools.partial(<function update_image at 0x7f6032e400>, Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07...', format='jpeg')),
 functools.partial(<function update_image at 0x7f6032e400>, Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07...', format='jpeg'))]

In [10]:
for (camera, observer) in zip(cameras, observers):
    try: camera.unobserve(observer, names='value')
    except ValueError: print('observer %s already unregistered' % observer)
    observers.remove(observer)
    camera.running = False
    
display(observers)

[functools.partial(<function update_image at 0x7f6032e400>, Image(value=b'\xff\xd8\xff\xdb\x00\x84\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07...', format='jpeg'))]

In [12]:
releaseCams = True

if releaseCams:
    for camera in cameras:
        camera.release()
        cameras.remove(camera)

display(cameras)

[]